In [1]:
from spark_utils import get_spark_session
import pyspark.sql.functions as F

spark = get_spark_session()
# run transform_latest.py if this don't exist
test_data = spark.read.parquet('data_transformed/amex-default-prediction/test_data_latest')
train_data = spark.read.parquet('data_transformed/amex-default-prediction/train_data_latest')
# run format_data.py if these don't exist
train_labels = spark.read.parquet('data/amex-default-prediction/train_labels')
sample_submission = spark.read.parquet('data/amex-default-prediction/sample_submission')

Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
22/06/20 12:37:13 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
22/06/20 12:37:14 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


In [2]:
%%time
from format_data import CATEGORICAL_VARIABLES
from encoder import CategoricalToIntegerEncoders

encs = CategoricalToIntegerEncoders(columns=CATEGORICAL_VARIABLES).fit(train_data)

CPU times: user 376 ms, sys: 36.9 ms, total: 413 ms
Wall time: 8.29 s


In [3]:
%%time
train_pdf = encs.transform(
    spark=spark,
    df=train_data.join(train_labels, on='customer_ID', how='inner')
).toPandas()

22/06/20 12:37:28 WARN package: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


CPU times: user 512 ms, sys: 296 ms, total: 808 ms
Wall time: 12 s


In [4]:
%%time
test_pdf = encs.transform(
    spark=spark,
    df=test_data
).toPandas()

CPU times: user 701 ms, sys: 542 ms, total: 1.24 s
Wall time: 9.29 s


In [5]:
from format_data import TARGET_VARIABLE, DATE_VARIABLES, ID_VARIABLES

feature_columns = [
    c for c in train_pdf.columns 
    if c not in [TARGET_VARIABLE,] + ID_VARIABLES + list(DATE_VARIABLES.keys())
]
', '.join(feature_columns)

'P_2, D_39, B_1, B_2, R_1, S_3, D_41, B_3, D_42, D_43, D_44, B_4, D_45, B_5, R_2, D_46, D_47, D_48, D_49, B_6, B_7, B_8, D_50, D_51, B_9, R_3, D_52, P_3, B_10, D_53, S_5, B_11, S_6, D_54, R_4, S_7, B_12, S_8, D_55, D_56, B_13, R_5, D_58, S_9, B_14, D_59, D_60, D_61, B_15, S_11, D_62, D_65, B_16, B_17, B_18, B_19, B_20, S_12, R_6, S_13, B_21, D_69, B_22, D_70, D_71, D_72, S_15, B_23, D_73, P_4, D_74, D_75, D_76, B_24, R_7, D_77, B_25, B_26, D_78, D_79, R_8, R_9, S_16, D_80, R_10, R_11, B_27, D_81, D_82, S_17, R_12, B_28, R_13, D_83, R_14, R_15, D_84, R_16, B_29, S_18, D_86, D_87, R_17, R_18, D_88, B_31, S_19, R_19, B_32, S_20, R_20, R_21, B_33, D_89, R_22, R_23, D_91, D_92, D_93, D_94, R_24, R_25, D_96, S_22, S_23, S_24, S_25, S_26, D_102, D_103, D_104, D_105, D_106, D_107, B_36, B_37, R_26, R_27, D_108, D_109, D_110, D_111, B_39, D_112, B_40, S_27, D_113, D_115, D_118, D_119, D_121, D_122, D_123, D_124, D_125, D_127, D_128, D_129, B_41, B_42, D_130, D_131, D_132, D_133, R_28, D_134, D_

In [6]:
import numpy as np

X_fit = train_pdf[feature_columns].reset_index(drop=True)
X_test = test_pdf[feature_columns].reset_index(drop=True)
print(X_fit.shape, X_test.shape)

y_fit = np.array(train_pdf[TARGET_VARIABLE])
print(np.unique(y_fit, return_counts=True))

(458913, 188) (924621, 188)
(array([0., 1.], dtype=float32), array([340085, 118828]))


In [7]:
from sklearn.model_selection import train_test_split

X_train, X_valid, y_train, y_valid = train_test_split(X_fit, y_fit)
X_train.shape, X_valid.shape, y_train.shape, y_valid.shape

((344184, 188), (114729, 188), (344184,), (114729,))

In [8]:
%%time
from evaluation import evaluate
from lightgbm import LGBMClassifier

m = LGBMClassifier().fit(
    X=X_train, y=y_train,
    categorical_feature=encs.columns_encoded,
)
# these are auto logged
score_train = m.score(X=X_train, y=y_train)
score_valid = m.score(X=X_valid, y=y_valid)
print(score_train, score_valid)

# some custom metrics
pred_train = m.predict_proba(X_train)[:, 1]
pred_valid = m.predict_proba(X_valid)[:, 1]
score_amex_train = evaluate(y_train, pred_train)
score_amex_valid = evaluate(y_valid, pred_valid)
print(score_amex_train, score_amex_valid)

/usr/local/lib/python3.8/dist-packages/lightgbm/basic.py:2065: UserWarning: Using categorical_feature in Dataset.
  _log_warning('Using categorical_feature in Dataset.')


0.904728284870883 0.9003390598715233
0.7972720896210039 0.7825648717550036
CPU times: user 1min 25s, sys: 588 ms, total: 1min 26s
Wall time: 13 s
